In [ ]:
import tempfile, shutil, os

In [ ]:
snapshot_path = os.environ.get('PROMETHEUS_SNAPSHOT_PATH', None)
bucket_name = os.environ.get('GCS_BUCKET', None)
prom_host = os.environ.get('PROMETHEUS_HOST', 'localhost')
prom_port = os.environ.get('PROMETHEUS_PORT', 9090)

In [ ]:
import http.client,datetime,os,requests, json

In [ ]:
def create_snapshot(https, host, port):
    conn = None
    try:
        if https:
            conn = http.client.HTTPSConnection(host, port=port)
        else:
            conn = http.client.HTTPConnection(host, port=port)
        conn.request( "POST", "/api/v1/admin/tsdb/snapshot")
        response = conn.getresponse()
    except e:
        print(e)
        return None
    if response.status != 200:
        print(f"Request failed! Status code: {response.status}")
        return None
    result = json.load(response)
    if conn != None:
        conn.close()
    return result['data']['name']

In [ ]:
snapshot_name = create_snapshot(False, prom_host, prom_port)

In [ ]:
tempdir = tempfile.mkdtemp()

In [ ]:
!scp -r {snapshot_path}/{snapshot_name} {tempdir}

In [ ]:
!gsutil -m cp -r {tempdir}/{snapshot_name} gs://{bucket_name}/{snapshot_name}

In [ ]:
shutil.rmtree(tempdir)